In [2]:
import pandas as pd
import numpy as np
import json
import requests
import tensorflow as tf
import subprocess


In [3]:
#This will start the serving app "servingapp.py" that will return the prediction of the next traffic matrix (one time step at a time)

subprocess.Popen("python ./servingapp.py")

<Popen: returncode: None args: 'python ./servingapp.py'>

In [4]:
file_name = "./Abilene-OD_pair.txt"
df = pd.read_csv(file_name)
data_columns = list(df.columns.values)
data_columns.remove('time')
data = df[data_columns].values
data = np.clip(data, 0.0, np.percentile(data.flatten(), 99))  # we use 99% as the threshold
df[data_columns] = data
timesteps=data.shape[0]
df['time']=df.index
times= df['time']
max_list = np.max(data, axis=0)
min_list = np.min(data, axis=0)
data = (data - min_list) / (max_list - min_list)
data[np.isnan(data)] = 0
data[np.isinf(data)] = 0
x_data = []
for i in range(timesteps ):
    x=data[i]
    x_data.append(x)

x_data = np.array(x_data)

x_data_2d=x_data.reshape((48096,12,12))
x_data_2d.shape


split_time=int(timesteps*0.8)
time_train = times[:split_time]
x_train = x_data_2d[:split_time]
time_valid = times[split_time:]
x_valid = x_data_2d[split_time:]


pred = x_valid[0:10]

In [5]:
pred.shape

(10, 12, 12)

In [8]:
# takes a 10*12*12 ip matrix
# the first dim (10),is the time dimension, the prediction needs 10 previous points/matrices in time.
# each 12*12 slice is a traffic matrix that contains the traffic between the IP repsented by the row incdice and the IP repsented by the column indice 

def get_prediction(matrix):
    data = {"data": matrix.tolist()}
    url = "http://localhost:5000/predict"
    return requests.post(url, json=data).json()['result']

In [9]:
get_prediction(pred)

[[[[0.0006681997328996658],
   [0.0035431080032140017],
   [0.0048817521892488],
   [0.001586447237059474],
   [0.0020697100553661585],
   [0.0010663646971806884],
   [0.00041140473331324756],
   [0.013828284107148647],
   [0.022555632516741753],
   [4.462548531591892e-06],
   [0.0014476882060989738],
   [0.0063233692198991776]],
  [[0.0023666168563067913],
   [0.14810225367546082],
   [0.02771497517824173],
   [0.057470329105854034],
   [0.13430996239185333],
   [0.06404172629117966],
   [0.04146197810769081],
   [0.058104317635297775],
   [0.042695432901382446],
   [0.0016498722834512591],
   [0.1446528136730194],
   [0.11873020231723785]],
  [[0.0014233049005270004],
   [0.09378872811794281],
   [0.3494775891304016],
   [0.03170483931899071],
   [0.06447447091341019],
   [0.08895116299390793],
   [0.0349503755569458],
   [0.07787106931209564],
   [0.04175998643040657],
   [0.10246382653713226],
   [0.06933742016553879],
   [0.13856902718544006]],
  [[0.005042676813900471],
   [0.090